<a href="https://colab.research.google.com/github/EZ6990/DeepLearning/blob/master/KaggleHomeDepot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!mkdir .kaggle
import json
token = {"username":"bgudltaldanny","key":"65b62f1a36c14c834b4e88037463a4f8"}
with open('.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)

In [7]:
# !cp .kaggle/kaggle.json ~/.kaggle/kaggle.json
!touch ~/.kaggle/kaggle.json
!ls -la ~/.kaggle/kaggle.json

-rw-r--r-- 1 root root 0 Feb  8 09:51 /root/.kaggle/kaggle.json


In [0]:
!cp .kaggle/kaggle.json ~/.kaggle/kaggle.json

In [9]:
!chmod 600 ~/.kaggle/kaggle.json

!pip uninstall -y kaggle
!pip install --upgrade pip
!pip install kaggle==1.5.6


!kaggle competitions download -c home-depot-product-search-relevance
!unzip home-depot-product-search-relevance.zip -d home-depot-product-search-relevance/
!unzip home-depot-product-search-relevance/train.csv.zip -d home-depot-product-search-relevance/
!unzip home-depot-product-search-relevance/test.csv.zip -d home-depot-product-search-relevance/

Found existing installation: kaggle 1.5.6
Uninstalling kaggle-1.5.6:
  Successfully uninstalled kaggle-1.5.6
Requirement already up-to-date: pip in /usr/local/lib/python3.6/dist-packages (20.0.2)
Processing /root/.cache/pip/wheels/01/3e/ff/77407ebac3ef71a79b9166a8382aecf88415a0bcbe3c095a01/kaggle-1.5.6-py3-none-any.whl
 59% 41.0M/69.4M [00:00<00:00, 29.8MB/s]
100% 69.4M/69.4M [00:00<00:00, 76.7MB/s]
Archive:  home-depot-product-search-relevance.zip
  inflating: home-depot-product-search-relevance/attributes.csv.zip  
  inflating: home-depot-product-search-relevance/product_descriptions.csv.zip  
  inflating: home-depot-product-search-relevance/relevance_instructions.docx  
  inflating: home-depot-product-search-relevance/sample_submission.csv.zip  
  inflating: home-depot-product-search-relevance/test.csv.zip  
  inflating: home-depot-product-search-relevance/train.csv.zip  
Archive:  home-depot-product-search-relevance/train.csv.zip
  inflating: home-depot-product-search-relevance/tra

In [0]:
import pandas as pd
import numpy as np
import math
import tensorflow as tf
from keras.utils import Sequence
from keras.models import Model
from keras.layers import Input, Flatten, Dense, Dropout,Conv1D, MaxPool1D,Concatenate, LSTM, Reshape, CuDNNLSTM, Embedding, BatchNormalization,Activation
from keras.optimizers import RMSprop
import keras.backend as K
from sklearn.pipeline import Pipeline,FeatureUnion,make_pipeline
from sklearn.preprocessing import FunctionTransformer,KBinsDiscretizer,OneHotEncoder
from sklearn.linear_model import LogisticRegression,LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score,cross_val_predict,cross_validate
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer,TfidfTransformer
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from keras.regularizers import l2
from scipy import spatial


In [0]:
path='./home-depot-product-search-relevance'
train=pd.read_csv(path+'/train.csv',encoding="Latin-1")

In [12]:
train.head()
train['norm_relevance'] = (train['relevance'] - 1) / 2
train.head()

,id,product_uid,product_title,search_term,relevance,norm_relevance
0,2,100001,Simpson Strong-Tie 12-Gauge Angle,angle bracket,3.00,1.000
1,3,100001,Simpson Strong-Tie 12-Gauge Angle,l bracket,2.50,0.750
2,9,100002,BEHR Premium Textured DeckOver 1-gal. #SC-141 ...,deck over,3.00,1.000
3,16,100005,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,rain shower head,2.33,0.665
4,17,100005,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,shower only faucet,2.67,0.835


In [0]:
max_product_title_len=train.product_title.map(lambda x: len(x)).max()
max_search_term=train.search_term.map(lambda x: len(x)).max()
max_input_len=max(max_product_title_len,max_search_term)

In [0]:
class MyCharGenerator(Sequence):
  def __init__(self,padding,df,batch_size=32,char_value=0):
    self.padding=padding
    self.char_value=char_value
    self.df=df
    self.batch_size=batch_size


  def __iter__(self):
    while(True):
      X1 = []
      X2 = []
      y = []

      for index,row in self.df.iterrows():
        y.append(row['norm_relevance'])
        st_list= [ord(c) for c in list(row['search_term'])]
        pt_list=[ord(c) for c in list(row['product_title'])]
        #X1.append(np.pad(st_list,(0,self.padding-len(st_list))))
        #X2.append(np.pad(pt_list,(0,self.padding-len(pt_list))))
        X1.append(np.reshape(np.pad(st_list,(0,self.padding-len(st_list))),(147,1)))
        X2.append(np.reshape(np.pad(pt_list,(0,self.padding-len(pt_list))),(147,1)))
        # assume there's one document per line, tokens separated by whitespace
        if (index + 1) % self.batch_size == 0:
          y=np.array(y)

          yield (np.array([X1,X2]),y)
          X1 = []
          X2 = []
          y=[]

      y=np.array(y)
      if(len(y) > 0):
        yield np.array([X1,X2]),y


In [0]:
class MyWordGenerator(Sequence):
  def __init__(self,padding,df,batch_size=32,char_value=0):
    self.padding=padding
    self.char_value=char_value
    self.df=df
    self.batch_size=batch_size
    self.words_dictonary = {}


  def __iter__(self):
    while(True):
      X1 = []
      X2 = []
      y = []

      for index,row in self.df.iterrows():
        y.append(row['norm_relevance'])
        st_list = []
        for word in row['search_term'].split(' '):
          if not (word in self.words_dictonary):
            self.words_dictonary[word] = len(self.words_dictonary)
          st_list.append(self.words_dictonary[word])
            
        pt_list = []
        for word in row['product_title'].split(' '):
          if not (word in self.words_dictonary):
            self.words_dictonary[word] = len(self.words_dictonary)
          pt_list.append(self.words_dictonary[word])

        X1.append(np.reshape(np.pad(st_list,(0,self.padding-len(st_list))),(self.padding,1)))
        X2.append(np.reshape(np.pad(pt_list,(0,self.padding-len(pt_list))),(self.padding,1)))
        # assume there's one document per line, tokens separated by whitespace
        if (index + 1) % self.batch_size == 0:
          y=np.array(y)

          yield (np.array([X1,X2]),y)
          X1 = []
          X2 = []
          y=[]

      y=np.array(y)
      if(len(y) > 0):
        yield np.array([X1,X2]),y

loss_index = 0
def rmse(y_true, y_pred):
  return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1)) 

def new_loss(y_actual,y_predicted):
  global loss_index
  global y_train
  global X1_train
  global X2_train

  cosine_loss = tf.keras.losses.CosineSimilarity(axis=1)
  loss = tf.constant(0.0)
  m = tf.constant(-0.00015)
  x = tf.constant(0.0)
  K.switch(tf.constant(y_train[x]))
  for i in range(32):
    if (True):
      loss = tf.add(loss,tf.subtract(tf.constant(1.0),
                                     cosine_loss(np.float32(X1_train[loss_index+i]),np.float32(X2_train[loss_index+i]))))
    else:
      loss = tf.add(loss,tf.maximum(tf.constant(0.0),
                                    tf.subtract(cosine_loss(np.float32(X1_train[loss_index+i]), np.float32(X2_train[loss_index+i])),m)))
  loss_index+=32
  with open('log.log', 'a') as file:
    json.dump(loss_index, file)
  tf.Print(loss,[loss],("%s" %loss_index))
  #return tf.add(K.sum(tf.multiply(tf.constant(0.0),y_predicted)),loss)
  return cosine_loss(y_actual,y_predicted)

# siames_model=network(max_input_len)
# siames_model.summary()
# siames_model.compile(loss=new_loss ,optimizer='adam',metrics=['mae','accuracy'])
# model_hist = siames_model.fit([X1_train,X2_train],y_train,verbose=2,epochs=3,batch_size=32,validation_data=([X1_test,X2_test],y_test))


In [0]:
myGen=MyCharGenerator(padding=147,df=train,batch_size=1)
it = iter(myGen)

X1 = []
X2 = []
Y = []
for time in range(len(train)):
  [X1_gen,X2_gen],y_gen = next(it)
  X1.append([X for X in X1_gen[0]])
  X2.append([X for X in X2_gen[0]])
  Y.append(y_gen[0])

X1_train,X1_test,X2_train,X2_test,y_train,y_test = train_test_split(X1,X2,Y,test_size=0.2)

X1_train = np.array(X1_train)
X2_train = np.array(X2_train)
y_train = np.array(y_train)
X1_test = np.array(X1_test)
X2_test = np.array(X2_test)
y_test = np.array(y_test)


In [0]:
def network(n_fetures,n_described=1):
  inp=Input(shape=(n_fetures,n_described))
  x = CuDNNLSTM(n_fetures)(inp)
  x = Reshape((n_fetures,1))(x)
  x=Conv1D(filters=32,kernel_size=10,activation='relu')(x)
  x=MaxPool1D()(x)
  x=Conv1D(filters=64,kernel_size=10,activation='relu')(x)
  x=MaxPool1D()(x)
  x=Conv1D(filters=128,kernel_size=10,activation='relu')(x)
  x=Flatten()(x)
  x=Dense(128,activation='relu')(x)
  model= Model(inp,x)

  search_term_input=Input(shape=(n_fetures,n_described),name='inp1')
  product_title_input=Input(shape=(n_fetures,n_described),name='inp2')

  search_term_encoded=model(search_term_input)
  product_title_encoded=model(product_title_input)


  x_connected=Concatenate()([search_term_encoded,product_title_encoded])
  x_connected=Dense(1,activation='sigmoid')(x_connected)
  
  
  connected_model=Model(inputs=[search_term_input,product_title_input],output=x_connected)
  connected_model.summary()

  return connected_model


In [18]:
siames_model=network(max_input_len)
siames_model.summary()
siames_model.compile(loss='msle' ,optimizer='adam',metrics=['mae','accuracy',rmse])





Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
inp1 (InputLayer)               (None, 147, 1)       0                                            
__________________________________________________________________________________________________
inp2 (InputLayer)               (None, 147, 1)       0                                            
__________________________________________________________________________________________________
model_1 (Model)                 (None, 128)          535336      inp1[0][0]                       
                                                                 inp2[0][0]                       
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, 256)          0           model_1[1][0]          

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("de...)`


In [19]:
model_hist = siames_model.fit([X1_train,X2_train],y_train,verbose=2,epochs=1,batch_size=20,validation_data=([X1_test,X2_test],y_test))

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 59253 samples, validate on 14814 samples
Epoch 1/1





 - 77s - loss: 0.0296 - mean_absolute_error: 0.2210 - acc: 0.2596 - rmse: 0.2210 - val_loss: 0.0295 - val_mean_absolute_error: 0.2171 - val_acc: 0.2525 - val_rmse: 0.2171


In [0]:
def get_query(X):
  return pd.DataFrame(X,columns=['query','title'])['query'].values
def get_title(X):
  return pd.DataFrame(X,columns=['query','title'])['title'].values

Naive_X = []
Naive_Y = []

for index,row in train.iterrows():
  Naive_Y.append(row['relevance'])
  Naive_X.append([''.join(list(row['search_term'])),''.join(list(row['product_title']))])

X_naive_train,X_naive_test,y_naive_train,y_naive_test = train_test_split(Naive_X,Naive_Y,test_size=0.2)

pipeline = Pipeline(
    [
     ('fetures',
         FeatureUnion(
             [
                 ('later',
                  Pipeline(
                      [
                          ('query',FunctionTransformer(get_query)),
                          ('vect', CountVectorizer(stop_words=[' '], max_df=0.5, max_features=5000,lowercase=False,analyzer='char'))
                      ]
                  )),
                 ('bins',
                  Pipeline(
                      [
                       ('title',FunctionTransformer(get_title)),
                       ('vect', CountVectorizer(stop_words=[' '], max_df=0.5, max_features=5000,lowercase=False,analyzer='char'))
                      ]
                  )
                 )
             ]
         )
      ),
     ('clf', LinearRegression()),
    ]
)
naive_model = cross_validate(pipeline, X_naive_train, y_naive_train, cv=5,return_estimator=True)

predictions = naive_model['estimator'][4].predict(X_naive_test)
mae = np.absolute((np.array(y_naive_test)-np.array(predictions))).mean()



rmse = math.sqrt(((np.array(y_naive_test)-np.array(predictions))**2).mean())
print("MAE: %.3f" %mae)
print("RMSE: %.3f" %rmse)

NameError: ignored

In [20]:
%%time

siames_model_ML = Model(siames_model.input,siames_model.layers[-2].output)
siames_model_ML.compile(loss='mse',optimizer='SGD')
siames_model_ML.summary()


ml_predictions_train = siames_model_ML.predict([X1_train,X2_train])
ml_predictions_test = siames_model_ML.predict([X1_test,X2_test])


# siames_model_ML=network(max_input_len)
# siames_model_ML.layers.pop(-1)
# siames_model_ML.summary()
# siames_model_ML.compile(optimizer='SGD',metrics=['mae','accuracy',rmse])
# model_hist_ml = siames_model_ML.fit([X1_train,X2_train],y_train,verbose=2,epochs=10,batch_size=20,validation_data=([X1_test,X2_test],y_test))




Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
inp1 (InputLayer)               (None, 147, 1)       0                                            
__________________________________________________________________________________________________
inp2 (InputLayer)               (None, 147, 1)       0                                            
__________________________________________________________________________________________________
model_1 (Model)                 (None, 128)          535336      inp1[0][0]                       
                                                                 inp2[0][0]                       
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, 256)          0           model_1[1][0]              

In [21]:
%%time

ml_knn_model = KNeighborsRegressor(n_jobs=16,weights='distance',algorithm='ball_tree',n_neighbors=3)
ml_knn_model.fit(ml_predictions_train,y_train*2+1)
ml_knn_predictions = ml_knn_model.predict(ml_predictions_test)

y_t=np.array(y_test)*2+1
mae_knn = np.absolute((y_t-np.array(ml_knn_predictions))).mean()
rmse_knn = math.sqrt(((y_t-np.array(ml_knn_predictions))**2).mean())

print("KNN MAE: %.3f" %mae_knn)
print("KNN RMSE: %.3f" %rmse_knn)


KNN MAE: 0.443
KNN RMSE: 0.539
CPU times: user 5min 56s, sys: 108 ms, total: 5min 56s
Wall time: 3min 9s


In [0]:
unique, counts = np.unique(ml_knn_predictions, return_counts=True)
print (unique)
print (counts)


[1.         1.165      1.20060037 1.31999395 1.33       1.33408628
 1.33986438 1.35122882 1.38271399 1.44666667 1.48315    1.49833468
 1.5        1.50370373 1.55333333 1.55666667 1.58596288 1.6020242
 1.62893108 1.64840037 1.65071362 1.65953642 1.66043196 1.665
 1.66666667 1.66673538 1.67       1.69939806 1.72860407 1.73288201
 1.73570589 1.7644921  1.76572408 1.7711161  1.77666667 1.77666667
 1.77674769 1.78       1.78087149 1.78454545 1.78690324 1.78880868
 1.79307365 1.80501487 1.83       1.83171668 1.83468029 1.835
 1.83870058 1.84271902 1.8457732  1.84690334 1.86023374 1.86494405
 1.87095401 1.88291093 1.88540315 1.88666667 1.89       1.89247975
 1.89907025 1.91166503 1.92803134 1.93527859 1.94360823 1.9562512
 1.96018265 1.96038914 1.96181059 1.97040687 1.97775777 1.97987284
 1.99138608 2.         2.         2.         2.0025     2.00416414
 2.00641327 2.01379121 2.0141841  2.01565451 2.02936778 2.03946262
 2.04580489 2.0469524  2.05525241 2.06358264 2.06637421 2.06789787
 2.0684

In [0]:
ml_predictions_train.shape

(59253, 256)

In [37]:
%%time


ml_rf_model = RandomForestRegressor(n_jobs=16,min_samples_leaf=2, max_depth =3,n_estimators=5)
ml_rf_model.fit(ml_predictions_train,y_train)
ml_rf_predictions = ml_rf_model.predict(ml_predictions_test)

y_t_rf=y_test*2+1

mae_rf = np.absolute((np.array(y_t_rf)-np.array(ml_rf_predictions*2+1))).mean()
rmse_rf = math.sqrt(((np.array(y_t_rf)-np.array(ml_rf_predictions*2+1))**2).mean())


print("RANDOM MAE: %.3f" %mae_rf)
print("RANDOM RMSE: %.3f" %rmse_rf)

RANDOM MAE: 0.437
RANDOM RMSE: 0.534
CPU times: user 949 ms, sys: 6.84 ms, total: 956 ms
Wall time: 628 ms


We explored the data and decided we need to do some preprocessing

we found some examples for words that ment the same thing but were wrote differently.

for example in the 'product title' column there is no 'inches' there is 'in.'
but in the 'search term' there is 'inches'

In [38]:
 print ('\'product_titles\' contains \'inches\'')
 display(train.loc[train['product_title'].str.contains(' inches ')])
 print ('\n\n\n\'search_terms\' contains \'inches\'')
 display(train.loc[train['search_term'].str.contains(' inches ')].head(5))

'product_titles' contains 'inches'


,id,product_uid,product_title,search_term,relevance,norm_relevance





'search_terms' contains 'inches'


,id,product_uid,product_title,search_term,relevance,norm_relevance
4015,12507,102193,Samsung 28.15 cu. ft. 4-Door French Door Refri...,33 inches wide samsung,2.67,0.835
13646,42206,110179,JELD-WEN Smooth 4-Panel Primed Molded Interior...,interior doors 82 inches in length,2.33,0.665
14917,46069,111389,Paslode 3 in. x 0.131-Gauge 30å¡ Brite Smooth ...,3 .5 inches paper tape collated,2.00,0.500
15609,48121,112069,Roberts Laminate Cutter for Cross Cutting up t...,metric to inches converter,1.67,0.335
17099,52792,113648,Masonite 30 in. x 80 in. Textured 6-Panel Holl...,interior doors 82 inches in length,2.67,0.835


In [0]:
import re
def preprocess_sentences(s):
  s = s.lower()
  s = re.sub(r"(\d+)'",r'\1in.',s)
  s = s.replace("inches","in.") 
  s = s.replace("inch","in.")
  s = s.replace(" in ","in. ") 
  s = s.replace(" in.","in.") 

  s = s.replace("''","ft.") 
  s = s.replace(" feet ","ft. ") 
  s = s.replace("feet","ft.") 
  s = s.replace("foot","ft.") 
  s = s.replace(" ft ","ft. ") 
  s = s.replace(" ft.","ft.") 

  s = s.replace(" pounds ","lb. ")
  s = s.replace(" pound ","lb. ") 
  s = s.replace("pound","lb.") 
  s = s.replace(" lb ","lb. ") 
  s = s.replace(" lb.","lb.") 
  s = s.replace(" lbs ","lb. ") 
  s = s.replace("lbs.","lb.") 

  s = s.replace(" x "," xby ")
  s = s.replace("*"," xby ")
  s = s.replace(" by "," xby")
  s = s.replace("x0"," xby 0")
  s = s.replace("x1"," xby 1")
  s = s.replace("x2"," xby 2")
  s = s.replace("x3"," xby 3")
  s = s.replace("x4"," xby 4")
  s = s.replace("x5"," xby 5")
  s = s.replace("x6"," xby 6")
  s = s.replace("x7"," xby 7")
  s = s.replace("x8"," xby 8")
  s = s.replace("x9"," xby 9")
  s = s.replace("0x","0 xby ")
  s = s.replace("1x","1 xby ")
  s = s.replace("2x","2 xby ")
  s = s.replace("3x","3 xby ")
  s = s.replace("4x","4 xby ")
  s = s.replace("5x","5 xby ")
  s = s.replace("6x","6 xby ")
  s = s.replace("7x","7 xby ")
  s = s.replace("8x","8 xby ")
  s = s.replace("9x","9 xby ")

  s = s.replace(" sq ft","sq.ft. ") 
  s = s.replace("sq ft","sq.ft. ")
  s = s.replace("sqft","sq.ft. ")
  s = s.replace(" sqft ","sq.ft. ") 
  s = s.replace("sq. ft","sq.ft. ") 
  s = s.replace("sq ft.","sq.ft. ") 
  s = s.replace("sq feet","sq.ft. ") 
  s = s.replace("square feet","sq.ft. ") 

  s = s.replace(" gallons ","gal. ") 
  s = s.replace(" gallon ","gal. ") 
  s = s.replace("gallons","gal.") 
  s = s.replace("gallon","gal.") 
  s = s.replace(" gal ","gal. ") 
  s = s.replace(" gal","gal.") 

  s = s.replace("ounces","oz.")
  s = s.replace("ounce","oz.")
  s = s.replace(" oz.","oz. ")
  s = s.replace(" oz ","oz. ")

  s = s.replace("centimeters","cm.")    
  s = s.replace(" cm.","cm.")
  s = s.replace(" cm ","cm. ")
  
  s = s.replace("milimeters","mm.")
  s = s.replace(" mm.","mm.")
  s = s.replace(" mm ","mm. ")
  
  s = s.replace("°","deg. ")
  s = s.replace("degrees","deg. ")
  s = s.replace("degree","deg. ")
  
  s = s.replace("volts","volt. ")
  s = s.replace("volt","volt. ")

  s = s.replace("watts","watt. ")
  s = s.replace("watt","watt. ")

  return s

In [40]:
processed_train = train.copy()
processed_train['search_term'] = processed_train['search_term'].map(lambda x:preprocess_sentences(x))
processed_train['product_title'] = processed_train['product_title'].map(lambda x:preprocess_sentences(x))

print ('\'product_titles\' contains \'inches\'')
display(processed_train.loc[processed_train['product_title'].str.contains(' in. ')])
print ('\n\n\n\'search_terms\' contains \'inches\'')
display(processed_train.loc[processed_train['search_term'].str.contains(' in. ')].head(5))

'product_titles' contains 'inches'


,id,product_uid,product_title,search_term,relevance,norm_relevance
18498,57112,115108,ready-strip 32oz. environmentally friendly dr...,adhesive slide strip,1.67,0.335
25119,77115,122211,bonded logic inc ultrasonic 12in. xby 12in. ac...,12in. ceiling tile,2.00,0.500
25120,77121,122211,bonded logic inc ultrasonic 12in. xby 12in. ac...,sound dampening,2.67,0.835
25121,77123,122211,bonded logic inc ultrasonic 12in. xby 12in. ac...,sounds panels,2.00,0.500
67581,202995,191353,mohawk home ink swirl cocoa 2ft. xby 8ft. runner,rug ruunners,2.00,0.500





'search_terms' contains 'inches'


,id,product_uid,product_title,search_term,relevance,norm_relevance
42161,128340,144629,liberty 20in. european self-closing drawer sli...,18.75 in. drawer slides,2.67,0.835
54368,164835,164764,kaleen matira blue 2ft. xby 3ft. indoor/outdoo...,"kaleen rugs, inc matira area rug 3 x",2.00,0.500
55418,167918,166665,kaleen matira blue 2ft. xby 3ft. indoor/outdoo...,"kaleen rugs, inc matira area rug 3 x",3.00,1.000
60397,182300,176124,kaleen matira blue 3ft. xby 5ft. indoor/outdoo...,"kaleen rugs, inc matira area rug 3 x",3.00,1.000


In [0]:
max_product_title_len=processed_train.product_title.map(lambda x: len(x.split(' '))).max()
max_search_term=processed_train.search_term.map(lambda x: len(x.split(' '))).max()
max_input_len=max(max_product_title_len,max_search_term,70)

In [0]:
myGen=MyWordGenerator(padding=max_input_len,df=processed_train,batch_size=1)
it = iter(myGen)

X1 = []
X2 = []
Y = []
for time in range(len(processed_train)):
  [X1_gen,X2_gen],y_gen = next(it)
  X1.append([X for X in X1_gen[0]])
  X2.append([X for X in X2_gen[0]])
  Y.append(y_gen[0])

X1_train,X1_test,X2_train,X2_test,y_train,y_test = train_test_split(X1,X2,Y,test_size=0.2)

X1_train = np.array(X1_train)
X2_train = np.array(X2_train)
y_train = np.array(y_train)
X1_test = np.array(X1_test)
X2_test = np.array(X2_test)
y_test = np.array(y_test)


In [57]:
def embedding_input(name, n_in, n_out, reg=1e-4):
    inp = Input(shape=(70,1), dtype='int64', name=name)
    x = Flatten()(inp)
    return inp, Embedding(n_in, n_out, input_length=70, embeddings_regularizer=l2(reg))(x)

query_inp,query_emb = embedding_input("query",len(processed_train.search_term.unique()),50)
title_inp,title_emb = embedding_input("title",len(processed_train.product_title.unique()),50)
x = Concatenate()([query_emb,title_emb])
x = Flatten()(x)
x= BatchNormalization()(x)
# x= Activation('relu')(x)
x = Dense(1,activation='relu')(x)
embedding_model = Model([query_inp, title_inp], x)
embedding_model.compile(optimizer='adam', loss='mae',metrics=['accuracy',rmse,'mae'])
embedding_model.summary()


embedding_model_hist = embedding_model.fit([X1_train,X2_train], y_train, batch_size=20, epochs=10, 
          validation_data=([X1_test,X2_test], y_test))

Model: "model_13"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
query (InputLayer)              (None, 70, 1)        0                                            
__________________________________________________________________________________________________
title (InputLayer)              (None, 70, 1)        0                                            
__________________________________________________________________________________________________
flatten_38 (Flatten)            (None, 70)           0           query[0][0]                      
__________________________________________________________________________________________________
flatten_39 (Flatten)            (None, 70)           0           title[0][0]                      
___________________________________________________________________________________________

In [73]:
inp=Input(shape=(70,50))
x = CuDNNLSTM(70)(inp)
x = Reshape((70,1))(x)
x=Conv1D(filters=32,kernel_size=10,activation='relu')(x)
x=MaxPool1D()(x)
x=BatchNormalization()(x)
x=Activation('tanh')(x)
x=Conv1D(filters=64,kernel_size=10,activation='relu')(x)
x=MaxPool1D()(x)
x=Conv1D(filters=128,kernel_size=10,activation='relu')(x)
x=BatchNormalization()(x)
x=Activation('tanh')(x)
x=Flatten()(x)
x=Dense(128,activation='relu')(x)
model= Model(inp,x)

search_term_encoded=model(query_emb)
product_title_encoded=model(title_emb)


x_connected=Concatenate()([search_term_encoded,product_title_encoded])
x_connected=Dense(1,activation='sigmoid')(x_connected)

siames_embedded_model = Model([query_inp,title_inp],x_connected)
siames_embedded_model.summary()
siames_embedded_model.compile(loss='mse' ,optimizer='adam',metrics=['mae','accuracy',rmse])

Model: "model_25"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
query (InputLayer)              (None, 70, 1)        0                                            
__________________________________________________________________________________________________
title (InputLayer)              (None, 70, 1)        0                                            
__________________________________________________________________________________________________
flatten_38 (Flatten)            (None, 70)           0           query[0][0]                      
__________________________________________________________________________________________________
flatten_39 (Flatten)            (None, 70)           0           title[0][0]                      
___________________________________________________________________________________________

In [74]:
word_model_hist = siames_embedded_model.fit([X1_train,X2_train],y_train,verbose=2,epochs=3,batch_size=32,validation_data=([X1_test,X2_test],y_test))

Train on 59253 samples, validate on 14814 samples
Epoch 1/3
 - 42s - loss: 0.0722 - mean_absolute_error: 0.2144 - acc: 0.2580 - rmse: 0.2144 - val_loss: 0.1652 - val_mean_absolute_error: 0.3043 - val_acc: 0.2585 - val_rmse: 0.3043
Epoch 2/3
 - 39s - loss: 0.0670 - mean_absolute_error: 0.2020 - acc: 0.2596 - rmse: 0.2020 - val_loss: 0.0950 - val_mean_absolute_error: 0.2285 - val_acc: 0.2585 - val_rmse: 0.2285
Epoch 3/3
 - 39s - loss: 0.0647 - mean_absolute_error: 0.1964 - acc: 0.2623 - rmse: 0.1964 - val_loss: 0.0796 - val_mean_absolute_error: 0.2271 - val_acc: 0.2585 - val_rmse: 0.2271


In [0]:
def get_query(X):
  return pd.DataFrame(X,columns=['query','title'])['query'].values
def get_title(X):
  return pd.DataFrame(X,columns=['query','title'])['title'].values

Naive_X = []
Naive_Y = []
for index,row in processed_train.iterrows():
  Naive_Y.append(row['relevance'])
  Naive_X.append([row['search_term'],row['product_title']])

X_naive_train,X_naive_test,y_naive_train,y_naive_test = train_test_split(Naive_X,Naive_Y,test_size=0.2)

pipeline = Pipeline(
    [
     ('fetures',
         FeatureUnion(
             [
                 ('later',
                  Pipeline(
                      [
                          ('query',FunctionTransformer(get_query)),
                          ('vect', CountVectorizer(stop_words=[' '], max_df=0.5, max_features=5000,lowercase=False))
                      ]
                  )),
                 ('bins',
                  Pipeline(
                      [
                       ('title',FunctionTransformer(get_title)),
                       ('vect', CountVectorizer(stop_words=[' '], max_df=0.5, max_features=5000,lowercase=False))
                      ]
                  )
                 )
             ]
         )
      ),
     ('clf', LinearRegression()),
    ]
)


naive_word_model = cross_validate(pipeline, X_naive_train ,y_naive_train, cv=3,return_estimator=True)

predictions = naive_word_model['estimator'][2].predict(X_naive_test)
mae = np.absolute((np.array(y_naive_test)-np.array(predictions))).mean()
rmse = math.sqrt(((np.array(y_naive_test)-np.array(predictions))**2).mean())
print("MAE: %.3f" %mae)
print("RMSE: %.3f" %rmse)

MAE: 0.427
RMSE: 0.538


In [75]:

siames_word_model_ML = Model(siames_embedded_model.input,siames_embedded_model.layers[-2].output)
siames_word_model_ML.compile(optimizer='SGD')
siames_word_model_ML.summary()


word_ml_predictions_train = siames_word_model_ML.predict([X1_train,X2_train])
word_ml_predictions_test = siames_word_model_ML.predict([X1_test,X2_test])




Model: "model_26"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
query (InputLayer)              (None, 70, 1)        0                                            
__________________________________________________________________________________________________
title (InputLayer)              (None, 70, 1)        0                                            
__________________________________________________________________________________________________
flatten_38 (Flatten)            (None, 70)           0           query[0][0]                      
__________________________________________________________________________________________________
flatten_39 (Flatten)            (None, 70)           0           title[0][0]                      
___________________________________________________________________________________________

In [84]:
%%time

ml_knn_word_model = KNeighborsRegressor(n_jobs=16,weights='distance',algorithm='kd_tree',n_neighbors=3)
ml_knn_word_model.fit(word_ml_predictions_train,y_train*2+1)
word_ml_knn_predictions = ml_knn_word_model.predict(word_ml_predictions_test)

y_t=np.array(y_test)*2+1
word_mae_knn = np.absolute((y_t-np.array(word_ml_knn_predictions))).mean()
word_rmse_knn = math.sqrt(((y_t-np.array(word_ml_knn_predictions))**2).mean())


print("KNN MAE: %.3f" %word_mae_knn)
print("KNN RMSE: %.3f" %word_rmse_knn)



KNN MAE: 0.438
KNN RMSE: 0.556
CPU times: user 9.69 s, sys: 0 ns, total: 9.69 s
Wall time: 6.27 s


In [85]:
%%time

ml_rf_word_model = RandomForestRegressor(n_jobs=16,min_samples_leaf=2, max_depth =3,n_estimators=5)
ml_rf_word_model.fit(word_ml_predictions_train,y_train)
word_ml_rf_predictions = ml_rf_word_model.predict(word_ml_predictions_test)
y_t_rf=y_test*2+1

word_mae_rf = np.absolute((y_t_rf-np.array(word_ml_rf_predictions*2+1))).mean()
word_rmse_rf = math.sqrt(((y_t_rf-np.array(word_ml_rf_predictions*2+1))**2).mean())

print("RANDOM MAE: %.3f" %word_mae_rf)
print("RANDOM RMSE: %.3f" %word_rmse_rf)


RANDOM MAE: 0.401
RANDOM RMSE: 0.497
CPU times: user 8.03 s, sys: 0 ns, total: 8.03 s
Wall time: 4.34 s
